In [1]:
from pymavlink import mavutil
import numpy
# Create a connection to the UDP endpoint
master = mavutil.mavlink_connection('udp:127.0.0.1:14551')
master.wait_heartbeat(timeout=10)
print("connection established")

connection established


In [2]:
def handle_mavlink_message(master):
    """
    Handle a MAVLink message.
    :param master: A MAVLink connection instance.
    """
    # Wait for a new message.
    msg = master.recv_match(blocking=True)

    # If a message has been received, handle it.
    if msg is not None:
        # Handle different message types.
        if msg.get_type() == 'ATTITUDE':
            print(f"Attitude: Roll={msg.roll}, Pitch={msg.pitch}, Yaw={msg.yaw}")
        elif msg.get_type() == 'GPS_RAW_INT':
            print(f"GPS: Lat={msg.lat / 1e7}, Lon={msg.lon / 1e7}, Alt={msg.alt / 1e3}m")
        elif msg.get_type() == 'SYS_STATUS':
            print(f"Battery: Voltage={msg.voltage_battery / 1000.0}V, Current={msg.current_battery / 100.0}A")


In [ ]:
while True:
    handle_mavlink_message(master)

In [5]:
from pymavlink import mavutil
from pymavlink.dialects.v20 import common as mavlink2

# Replace with your connection string
master = mavutil.mavlink_connection("COM3",baud=57600)

# Wait for the heartbeat message to ensure the connection is established
master.wait_heartbeat()

# Clear any existing missions
master.mav.mission_clear_all_send(master.target_system, master.target_component)

# Define the mission items
mission_items = [
    mavlink2.MAVLink_mission_item_message(
        0,  # target_system
        0,  # target_component
        0,  # seq
        mavlink2.MAV_FRAME_GLOBAL_RELATIVE_ALT,  # frame
        mavlink2.MAV_CMD_NAV_WAYPOINT,  # command
        0,  # current
        1,  # autocontinue
        0, 0, 0, 0,  # params 1-4
        34.052235,  # x (latitude)
        -118.243683,  # y (longitude)
        10  # z (altitude)
    )
    # Add more mission items as needed
]

# Send the mission count
master.mav.mission_count_send(master.target_system, master.target_component, len(mission_items))

# Send the mission items
for i, item in enumerate(mission_items):
    master.mav.send(item)
    # Wait for the vehicle to acknowledge the mission item
    master.recv_match(type='MISSION_ACK', blocking=True)

# Set the first mission item as the active mission item
master.mav.mission_set_current_send(master.target_system, master.target_component, 0)


master.mav.command_long_send(
    master.target_system,
    master.target_component,
    mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
    0,
    1, 0, 0, 0, 0, 0, 0)

## Test Sending Data

In [ ]:
mavlink_connection = mavutil.mavlink_connection(device='udpout:127.0.0.1:14550', baudrate=57600)
mavlink_connection.wait_heartbeat()

## Create Mission Pattern

In [ ]:


from pymavlink import mavutil
import time

# Replace with the serial port and baudrate that your Pixhawk is connected to
connection_string = 'COM6'
baudrate = 57600

# Connect to the Vehicle
master = mavutil.mavlink_connection(connection_string, baud=baudrate)
master.wait_heartbeat()

# Replace with your desired landing coordinates
landing_lat = 42.978990
landing_lon = -81.144590
landing_alt = 30  # altitude in meters

# Clear any existing missions
master.waypoint_clear_all_send()

# Set mission count to 1 (for landing)
master.waypoint_count_send(1)

# Wait for the request for the waypoint
msg = master.recv_match(type=['MISSION_REQUEST'], blocking=True)

# Send landing waypoint
master.mav.mission_item_send(
    master.target_system,
    master.target_component,
    msg.seq,
    mavutil.mavlink.MAV_FRAME_GLOBAL_RELATIVE_ALT,
    mavutil.mavlink.MAV_CMD_NAV_LAND,
    0,  # current
    1,  # autocontinue
    0, 0, 0,  # params 1-3 (not used)
    landing_lat,
    landing_lon,
    landing_alt
)

# Wait for mission acknowledgement
master.recv_match(type=['MISSION_ACK'], blocking=True)

# Set mode to AUTO to start the mission
master.mav.set_mode_send(
    master.target_system,
    mavutil.mavlink.MAV_MODE_AUTO_ARMED,
    base_mode=mavutil.mavlink.MAV_MODE_FLAG_CUSTOM_MODE_ENABLED
)

# Arm the vehicle
master.mav.command_long_send(
    master.target_system,
    master.target_component,
    mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
    0,  # confirmation
    1,  # arm
    0, 0, 0, 0, 0, 0  # unused parameters
)

# Start the mission
master.mav.command_long_send(
    master.target_system,
    master.target_component,
    mavutil.mavlink.MAV_CMD_MISSION_START,
    0,  # confirmation
    0, 0, 0, 0, 0, 0  # unused parameters
)

print("Mission started, vehicle is landing...")


In [ ]:
# 